In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sqlite3
import numpy as np
import random
import urllib.request
import time
import os

In [2]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select Livsmedelsnamn, Livsmedelsnummer from livs limit 2000')
for row in rows:
        result.append(row)

#rows = np.array(result)
#rows = rows[970:]
samples = random.sample(range(0, 1999), 3)
sampledFoodItems = []
#print (type(sampledFoodItems))

for index in samples:
    sampledFoodItems.append(result[index])

sampledFoodItems = np.array(sampledFoodItems)
print (samples, sampledFoodItems)

conn.close()

[364, 13, 1395] [['Tomater gröna syltade' '421']
 ['Lättmargarin m växtsterol 7,5% fett 35% berik typ Becel proactiv' '18']
 ['Smördeg gräddad' '1640']]


In [3]:
def download_page(url): #urllib library for Extracting web pages
    try:
        headers = {}
        headers['User-Agent'] = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
        req = urllib.request.Request(url, headers = headers)
        resp = urllib.request.urlopen(req)
        respData = str(resp.read())
        return respData
    except Exception as e:
        print(str(e))

In [4]:
#Finding 'Next Image' from the given raw page
def _images_get_next_item(s):
    print (type(s))
    start_line = s.find('rg_di')
    if start_line == -1:    #If no links are found then give an error!
        end_quote = 0
        link = "no_links"
        return link, end_quote
    else:
        start_line = s.find('"class="rg_meta"')
        start_content = s.find('"ou"',start_line+1)
        end_content = s.find(',"ow"',start_content+1)
        content_raw = str(s[start_content+6:end_content-1])
        return content_raw, end_content

In [5]:
def _images_get_all_items(page):
    items = []
    while True:
        item, end_content = _images_get_next_item(page)
        if item == "no_links":
            break
        else:
            items.append(item)      #Append all the links in the list named 'Links'
            time.sleep(0.1)        #Timer could be used to slow down the request for image downloads
            page = page[end_content:]
    return items


In [6]:
t0 = time.time()   #start the timer
keywords = sampledFoodItems
#Prepare keywords
for i, item in enumerate(keywords):
    keywords[i][0]=keywords[i][0].replace(' ','%20')
print (keywords)

[['Tomater%20gröna%20syltade' '421']
 ['Lättmargarin%20m%20växtsterol%207,5%%20fett%2035%%20berik%20typ%' '18']
 ['Smördeg%20gräddad' '1640']]


In [8]:
    #Download Image Links
    for index, item in enumerate(keywords):
        iteration = "Item no.: " + str(index) + " -->" + " Item name = " + str(item[0])
        print (iteration)
        print ("Evaluating...")
        print(item)

         #make a search keyword  directory
        try:
            os.makedirs(item[1])
        except OSError as e:
            if e.errno != 17:
                raise
            # time.sleep might help here
            pass


        url = 'https://www.google.se/search?q=' + item[0] +'&espv=2&biw=1366&bih=667&site=webhp&source=lnms&tbm=isch&sa=X&ei=XosDVaCXD8TasATItgE&ved=0CAcQ_AUoAg'
        print(url)
        #raw_html =  download_page(url)
        #print(raw_html)
        
        #time.sleep(0.1)
        #imageLinks = (_images_get_all_items(raw_html)) # + items

        #print ("Image Links = "+str(imageLinks))
        #print ("Total Image Links = "+str(len(imageLinks)))
        #print ("\n")


Item no.: 0 --> Item name = Tomater%20gröna%20syltade
Evaluating...
['Tomater%20gröna%20syltade' '421']
https://www.google.se/search?q=Tomater%20gröna%20syltade&espv=2&biw=1366&bih=667&site=webhp&source=lnms&tbm=isch&sa=X&ei=XosDVaCXD8TasATItgE&ved=0CAcQ_AUoAg
Item no.: 1 --> Item name = Lättmargarin%20m%20växtsterol%207,5%%20fett%2035%%20berik%20typ%
Evaluating...
['Lättmargarin%20m%20växtsterol%207,5%%20fett%2035%%20berik%20typ%' '18']
https://www.google.se/search?q=Lättmargarin%20m%20växtsterol%207,5%%20fett%2035%%20berik%20typ%&espv=2&biw=1366&bih=667&site=webhp&source=lnms&tbm=isch&sa=X&ei=XosDVaCXD8TasATItgE&ved=0CAcQ_AUoAg
Item no.: 2 --> Item name = Smördeg%20gräddad
Evaluating...
['Smördeg%20gräddad' '1640']
https://www.google.se/search?q=Smördeg%20gräddad&espv=2&biw=1366&bih=667&site=webhp&source=lnms&tbm=isch&sa=X&ei=XosDVaCXD8TasATItgE&ved=0CAcQ_AUoAg


In [14]:
def downloadImages(keyword):
    k=0
    numberOfImages = 5 # Max is len(items)
    errorCount=0
    while(k<numberOfImages):
        import urllib.request
        import urllib.error

        try:
            req = urllib.request.Request(items[k], headers={"User-Agent": "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1312.27 Safari/537.17"})
            response = urllib.request.urlopen(req,None,15)
            output_file = open(keyword[1]+"/"+str(k+1)+".jpg",'wb')

            data = response.read()
            output_file.write(data)
            response.close();

            print("completed ====> "+str(k+1))

            k=k+1;

        except urllib.error.HTTPError as e:  #If there is any HTTPError

            errorCount+=1
            print("HTTPError"+str(k))
            k=k+1;
        except urllib.error.URLError as e:

            errorCount+=1
            print("URLError "+str(k))
            k=k+1;


In [15]:
#This allows you to write all the links into a test file. This text file will be created in the same directory as your code. You can comment out the below 3 lines to stop writing the output to the text file.
#info = open('output.txt', 'a')        #Open the text file called database.txt
#info.write(str(i) + ': ' + str(search_keyword[i-1]) + ": " + str(items) + "\n\n\n")         #Write the title of the page
#info.close()                            #Close the file

t1 = time.time()    #stop the timer
total_time = t1-t0   #Calculating the total time required to crawl, find and download all the links of 60,000 images
print("Total time taken: "+str(total_time)+" Seconds")
print ("Starting Download...")

## To save imges to the same directory
# IN this saving process we are just skipping the URL if there is any error
print(keywords[0])
downloadImages(keywords[0])
i = i+1

print("\n")
print("Everything downloaded!")
print("\n"+str(errorCount)+" ----> total Errors")


Total time taken: 357.4313762187958 Seconds
Starting Download...
['Tomater%20gröna%20syltade' '421']


NameError: name 'items' is not defined